In [1]:
import pandas as pd
from urllib.parse import unquote

In [2]:

def title_parse(title):
    title = unquote(title)
    return title


In [3]:

columns = ['article','category']
df = pd.read_csv('wikispeedia_paths-and-graph/categories.tsv',delimiter='\t',encoding='UTF-8',skiprows=12,header=None,names=columns)
df['article']= df.article.apply(title_parse)


In [4]:
columns = ['category','ID']
df2 = pd.read_csv('category-ids.csv',names=columns)
df2

,category,ID
0,subject,C0001
1,subject.Art,C0002
2,subject.Business_Studies,C0003
3,subject.Citizenship,C0004
4,subject.Countries,C0005
...,...,...
141,subject.Science.Physics.Electricity_and_Electr...,C0142
142,subject.Science.Physics.General_Physics,C0143
143,subject.Science.Physics.Space_Astronomy,C0144
144,subject.Science.Physics.Space_transport,C0145


In [5]:
categories = list((df2['category']))
ID = list((df2['ID']))
cat_ID_dict = dict()
cat_child_dict = dict()

for i in range(len(categories)):
    cat_ID_dict[categories[i]]=ID[i]
    cat_child_dict[ID[i]]=list()
    
for categ in categories:
    
    categ_split = categ.split('.')
    temp = "subject"
    temp_ID = cat_ID_dict[temp]
    
    for i in range(len(categ_split)):
        
        if i!=0:
            temp_next = temp + "." + categ_split[i]
            temp_next_ID = cat_ID_dict[temp_next]
            #print(temp_next,temp_next_ID,temp,temp_ID)
            cat_child_dict[temp_ID].append(temp_next_ID)
            temp = temp_next
            temp_ID = temp_next_ID
            
for i in range(len(ID)):
    cat_child_dict[ID[i]]=set(cat_child_dict[ID[i]])
    

In [6]:
columns =['categories', 'paths','visited','shortest_paths','shortest_visited']
df3 = pd.read_csv('category-paths.csv',names=columns)

categories = list(df3['categories'])
paths = list(df3['paths'])
visited = list(df3['visited'])
shortest_paths = list(df3['shortest_paths'])
shortest_visited = list(df3['shortest_visited'])
df3

,categories,paths,visited,shortest_paths,shortest_visited
0,C0001,70,70,69,69
1,C0002,0,0,0,0
2,C0003,0,0,0,0
3,C0004,0,0,0,0
4,C0005,28375,41955,6736,7041
...,...,...,...,...,...
141,C0142,1277,1744,575,575
142,C0143,3910,5588,1499,1516
143,C0144,4631,7519,3497,3524
144,C0145,1227,1999,836,844


In [7]:
l0=[]
l1=[]
l2=[]
l3=[]

categories=list(set(df['category']))

for categ in categories:
    
    categ_split = categ.split('.')
    temp = ""
    for i in range(len(categ_split)):
        temp = temp + "." + categ_split[i]
        temp_s = temp[1:].split('.')
            
        if len(temp_s)==1:
            l0.append(temp[1:])
        elif len(temp_s)==2:
            l1.append(temp[1:])
        elif len(temp_s)==3:
            l2.append(temp[1:])
        elif len(temp_s)==4:
            l3.append(temp[1:])
            
l0 =  list(set(l0))
l2 =  list(set(l2))
l3 =  list(set(l3))
l1 =  list(set(l1))

ll = l2 + l1 + l0


In [8]:
categories = list((df2['ID']))
for parent in ll:
    parent_ID = cat_ID_dict[parent]
    
    for child in cat_child_dict[parent_ID]:
        idx = int(parent_ID[1:])-1
        c_idx = int(child[1:])-1
        paths[idx] += paths[c_idx]
        visited[idx] += visited[c_idx]
        shortest_paths[idx] += shortest_paths[c_idx]
        shortest_visited[idx] += shortest_visited[c_idx]
    

In [9]:
df4 = pd.DataFrame(list(zip(categories,paths,visited,shortest_paths,shortest_visited)), columns =['categories', 'paths','visited','shortest_paths','shortest_visited']) 
df4

,categories,paths,visited,shortest_paths,shortest_visited
0,C0001,281778,378168,119603,120946
1,C0002,1430,2005,737,742
2,C0003,4232,5100,1782,1795
3,C0004,14724,18146,4449,4483
4,C0005,28375,41955,6736,7041
...,...,...,...,...,...
141,C0142,1277,1744,575,575
142,C0143,3910,5588,1499,1516
143,C0144,4631,7519,3497,3524
144,C0145,1227,1999,836,844


In [10]:
df4.to_csv('category-subtree-paths.csv',index=False,header=None)